In [1]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)
soundstream.load("content/soundstream/results3/soundstream.9000.pt")
soundstream.to("cuda")



2023-01-10 16:10:57 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-10 16:10:57.649162: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 16:10:58.196649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-10 16:10:58.196701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-10 16:10:58.196706: W tensorflow/compiler/tf

SoundStream(
  (encoder): Sequential(
    (0): CausalConv1d(
      (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
    )
    (1): Sequential(
      (0): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (1): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (2): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(9,))
   

In [2]:
import sys
import torch
from importlib import reload
import network_models.soundstream_lstm.LSTM_dataset as lds
import torch.utils.data as data_utils
#from network_models.soundstream_lstm.LSTM_dataset  import AudioEmotionTessDataset
reload(lds)


tess_dataset = lds.AudioEmotionTessDataset("../tess")
#encoder_dataset = lds.NewAudioEmotionTessDataset("../tess", soundstream)
encoded_dataset = lds.AudioEmotionTessSoundStreamEncodedDataset(dataSet= tess_dataset, soundStream=soundstream)

In [3]:
import importlib
from pathlib import Path
import torch
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundstream_lstm.small_model as sm
importlib.reload(sm)


import network_models.soundstream_lstm.soundstream_datacollator as datacollator
importlib.reload(datacollator)
data_collator = datacollator.SoundStreamDataCollator(200)

model = sm.EmotionClassifierSevenEmos().to("cuda")

torch.Size([])


In [4]:
from torch.utils.data import Subset
import numpy as np
from network_models.soundstream_lstm.soundstream_datacollator import SoundstreamModelTrainer
from transformers import TrainingArguments, EvalPrediction

is_regression = False
from sklearn.model_selection import train_test_split

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    trainSet = Subset(dataset, train_idx)
    valSet = Subset(dataset, val_idx)
    return trainSet, valSet

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=0)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


train_dataset, eval_dataset = train_val_dataset(encoded_dataset, val_split=0.25)


Using cuda_amp half precision backend


In [6]:
import torch.optim as optim

learning_rate = 1e-4
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
evalLoader = torch.utils.data.DataLoader(eval_dataset, batch_size=10,
                                          shuffle=True, num_workers=2)
train_dataset[1]

(array([[ 0.30641523, -0.30414137, -0.3276682 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.40829998, -0.06142293, -0.00129516, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.08494987,  0.24296378, -0.02997825, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.32030046,  0.27021587,  0.36861932, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.02692641, -0.13035715, -0.1913992 , ...,  0.        ,
          0.        ,  0.        ],
        [-0.07453994, -0.16956666, -0.19240046, ...,  0.        ,
          0.        ,  0.        ]], dtype=float32),
 tensor([0., 0., 0., 0., 1., 0., 0.]))

In [7]:
import gc
from torchvision.transforms import Lambda
from torch.utils.data import DataLoader
from torch import nn
from sklearn.preprocessing import OneHotEncoder# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

def encodeLabel(label):
    return enc.fit_transform(label).toarray()


loss_fn = nn.CrossEntropyLoss()

learning_rate = 1e-3
batch_size = 10
epochs = 100
gc.collect()
torch.cuda.empty_cache()


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=2)
test_dataloader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.reshape(-1, 512*400).to("cuda")
        # Compute prediction and loss
        pred = model(X)
        y = y.to("cuda")

        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.reshape(-1, 512*400).to("cuda")
            y = y.to("cuda")
            pred = model(X)

            test_loss += loss_fn(pred, y.to("cuda")).item()
            y = torch.tensor([a.nonzero() for a in y]).to("cuda")
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.937051  [    0/ 2100]
loss: 1.952227  [ 1000/ 2100]
loss: 1.939513  [ 2000/ 2100]
Test Error: 
 Accuracy: 15.9%, Avg loss: 1.946134 

Epoch 2
-------------------------------
loss: 1.931537  [    0/ 2100]
loss: 1.943762  [ 1000/ 2100]
loss: 1.923206  [ 2000/ 2100]
Test Error: 
 Accuracy: 17.0%, Avg loss: 1.944619 

Epoch 3
-------------------------------
loss: 1.925678  [    0/ 2100]
loss: 1.934024  [ 1000/ 2100]
loss: 1.904628  [ 2000/ 2100]
Test Error: 
 Accuracy: 17.3%, Avg loss: 1.943027 

Epoch 4
-------------------------------
loss: 1.919361  [    0/ 2100]
loss: 1.923257  [ 1000/ 2100]
loss: 1.886316  [ 2000/ 2100]
Test Error: 
 Accuracy: 18.3%, Avg loss: 1.941338 

Epoch 5
-------------------------------
loss: 1.912491  [    0/ 2100]
loss: 1.911494  [ 1000/ 2100]
loss: 1.869779  [ 2000/ 2100]
Test Error: 
 Accuracy: 19.7%, Avg loss: 1.939552 

Epoch 6
-------------------------------
loss: 1.904943  [    0/ 2100]
loss: 1.898868  [ 10

In [41]:
import torchaudio
import torch

# trainer.train()
#torch.save(model.state_dict(), 'content/classifierSS/model_weights.pth')
audio = torchaudio.load("../Stimuli_Intensitätsmorphs/nf01_hap_w01_o_100_70dB.wav")

data = soundstream.encoder(audio[0].to("cuda"))
data = torch.nn.functional.pad(data, (0, 400 - data.shape[1], 0, 0))
pred = model(data.flatten())
print(pred)
encoded_dataset.getEmotionFromId(3)

tensor([0.0598, 0.0175, 0.1067, 0.7351, 0.0032, 0.0755, 0.0023],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


'happy'